In [110]:
import pandas as pd
from math import sqrt
from sklearn import datasets, svm, metrics
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression, LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler , StandardScaler, Imputer, LabelEncoder


In [137]:
diamonds =  pd.read_csv('./diamonds-datamad1019/data.csv/data.csv')

In [138]:
color = {
    "D": 7,
    "E": 6,
    "F": 5,
    "G": 4,
    "H": 3,
    "I": 2,
    "J": 1
}

cut = {
    "Ideal": 5,
    "Premium": 4,
    "Very Good": 3,
    "Good": 2,
    "Fair":1
}

clarity = {
    "IF": 8,
    "VVS1": 7,
    "VVS2": 6,
    "VS1": 5,
    "VS2": 4,
    "SI1": 3,
    "SI2": 2,
    "I1": 1
}

In [139]:
for i in range(len(diamonds)):
    co = color[diamonds.loc[i, "color"]]
    cu = cut[diamonds.loc[i, "cut"]]
    cl = clarity[diamonds.loc[i, "clarity"]]
    diamonds.loc[i, "color_num"] = co
    diamonds.loc[i, "cut_num"] = cu
    diamonds.loc[i, "clarity_num"] = cl

In [140]:
diamonds.head()

,carat,cut,color,clarity,depth,table,x,y,z,price,color_num,cut_num,clarity_num
0,2.26,Ideal,G,SI2,61.9,57.0,8.44,8.36,5.20,12831,4.0,5.0,2.0
1,2.43,Very Good,H,SI2,63.2,57.0,8.56,8.50,5.39,16170,3.0,3.0,2.0
2,0.80,Premium,F,SI2,61.0,57.0,6.03,6.01,3.67,2797,5.0,4.0,2.0
3,0.40,Ideal,F,I1,63.3,60.0,4.68,4.64,2.95,630,5.0,5.0,1.0
4,0.31,Ideal,G,VS2,61.6,55.0,4.39,4.37,2.70,698,4.0,5.0,4.0


In [112]:
label_cut = LabelEncoder()
label_color = LabelEncoder()
label_clarity = LabelEncoder()


diamonds['cut'] = label_cut.fit_transform(diamonds['cut'])
diamonds['color'] = label_color.fit_transform(diamonds['color'])
diamonds['clarity'] = label_clarity.fit_transform(diamonds['clarity'])

In [113]:
#diamonds = diamonds[(diamonds[['x','y','z']] != 0).all(axis=1)]

In [141]:
diamonds.drop(columns=['cut','color','clarity'], inplace=True)

In [142]:
diamonds['volume'] = diamonds['x']*diamonds['y']*diamonds['z']
diamonds.drop(['x','y','z'],axis=1,inplace=True)

In [143]:
diamonds.head()

,carat,depth,table,price,color_num,cut_num,clarity_num,volume
0,2.26,61.9,57.0,12831,4.0,5.0,2.0,366.903680
1,2.43,63.2,57.0,16170,3.0,3.0,2.0,392.176400
2,0.80,61.0,57.0,2797,5.0,4.0,2.0,133.001901
3,0.40,63.3,60.0,630,5.0,5.0,1.0,64.059840
4,0.31,61.6,55.0,698,4.0,5.0,4.0,51.797610


In [144]:
X = diamonds[["carat","color_num","cut_num","clarity_num"]]
#X = diamonds[["carat","color","cut","clarity"]]

In [145]:
y = diamonds.price

In [146]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.1)

In [147]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [148]:
models = {
    #"KNN": KNeighborsClassifier(n_neighbors=5),
    #"svm": LinearSVC(),
    #"logistic": LogisticRegression(solver='lbfgs', max_iter=2000),
    "forest": RandomForestClassifier()
}

for modelName, model in models.items():
    print(f"Training model: {modelName}")
    model.fit(X_train, y_train)

Training model: forest


/home/mariobru/.local/lib/python3.6/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [149]:
d = {modelName:model.predict(X_test) for modelName, model in models.items()}

df = pd.DataFrame(d)
y_test.reset_index(inplace=True, drop=True)
df["gt"] = y_test
df

,forest,gt
0,2398,2939
1,707,687
2,4586,4851
3,871,1117
4,4480,4608
...,...,...
4041,6932,5641
4042,5396,5170
4043,1250,1122
4044,3774,3407


In [150]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from IPython.core.display import display, HTML

switchColor = lambda value: f"<b style=\"color:orange\">{round(value,2)}</b>" if value < 0.8 else f"<b style=\"color:green\">{round(value,2)}</b>"
printBonito = lambda label,val: display(HTML(f"<span style=\"padding-left:20px\">The {label} is: {switchColor(val)}</span>"))

for modelName, model in models.items():    
    print(f"Evaluating model [{modelName}]:")
    printBonito("Accuracy", accuracy_score(df["gt"],df[modelName]))
    printBonito("Precision", precision_score(df["gt"],df[modelName],average='weighted'))
    printBonito("Recall", recall_score(df["gt"],df[modelName],average='weighted'))

Evaluating model [forest]:


/home/mariobru/.local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


/home/mariobru/.local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [151]:
diamonds_test =  pd.read_csv('./diamonds-datamad1019/test.csv')

In [132]:
label_cut = LabelEncoder()
label_color = LabelEncoder()
label_clarity = LabelEncoder()


diamonds_test['cut'] = label_cut.fit_transform(diamonds_test['cut'])
diamonds_test['color'] = label_color.fit_transform(diamonds_test['color'])
diamonds_test['clarity'] = label_clarity.fit_transform(diamonds_test['clarity'])

In [152]:
for i in range(len(diamonds_test)):
    co = color[diamonds_test.loc[i, "color"]]
    cu = cut[diamonds_test.loc[i, "cut"]]
    cl = clarity[diamonds_test.loc[i, "clarity"]]
    diamonds_test.loc[i, "color_num"] = co
    diamonds_test.loc[i, "cut_num"] = cu
    diamonds_test.loc[i, "clarity_num"] = cl

In [154]:
diamonds_test.drop(columns=['cut','color','clarity'], inplace=True)

In [155]:
#diamonds_test = diamonds_test[(diamonds_test[['x','y','z']] != 0).all(axis=1)]

In [156]:
X = diamonds_test[["carat","color_num","cut_num","clarity_num"]]

In [157]:
d = {modelName:model.predict(X) for modelName, model in models.items()}
print(d)

df = pd.DataFrame(d)


{'forest': array([15813, 17590, 17590, ..., 17590,  8979, 16469])}


In [158]:
#df.drop(columns=['KNN'], inplace=True)
df.rename(columns={"forest": "price"}, inplace=True)
df

,price
0,15813
1,17590
2,17590
3,17590
4,15813
...,...
13480,17590
13481,15813
13482,17590
13483,8979


In [159]:
df.to_csv('./diamonds-datamad1019/submission.csv')